## LangChain: Memory

### Outline
- ConversationBufferMemory
- ConversationBufferWindowMemory
- ConversationTokenBufferMemory
- ConversationSummaryMemory

### ConversationBufferMemory

In [1]:
# Import required packages
import os
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# Define Azure OpenAI endpoint parameters
OPENAI_API_VERSION = "2023-07-01-preview"
OPENAI_DEPLOYMENT = os.getenv("OPENAI_API_DEPLOY") # Set AOAI deployment name as env variable

In [2]:
# Instantiation of the ConversationChain class
llm = AzureChatOpenAI(
    openai_api_version=OPENAI_API_VERSION,
    deployment_name=OPENAI_DEPLOYMENT,
    temperature=0.0
)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [3]:
conversation.predict(input="Hi, my name is Andrew")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [4]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI:

> Finished chain.


'1+1 is equal to 2.'

In [5]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI:

> Finished chain.


'Your name is Andrew.'

In [6]:
print(memory.buffer)

Human: Hi, my name is Andrew
AI: Hello Andrew! It's nice to meet you. How can I assist you today?
Human: What is 1+1?
AI: 1+1 is equal to 2.
Human: What is my name?
AI: Your name is Andrew.


In [7]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Andrew\nAI: Hello Andrew! It's nice to meet you. How can I assist you today?\nHuman: What is 1+1?\nAI: 1+1 is equal to 2.\nHuman: What is my name?\nAI: Your name is Andrew."}

In [8]:
memory = ConversationBufferMemory()

In [9]:
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [10]:
print(memory.buffer)

Human: Hi
AI: What's up


In [11]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [12]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [13]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

### ConversationBufferWindowMemory

In [14]:
from langchain.memory import ConversationBufferWindowMemory

In [15]:
memory = ConversationBufferWindowMemory(k=1)

In [16]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

In [17]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [18]:
llm = AzureChatOpenAI(
    openai_api_version=OPENAI_API_VERSION,
    deployment_name=OPENAI_DEPLOYMENT,
    temperature=0.0
)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [19]:
conversation.predict(input="Hi, my name is Andrew")

"Hello Andrew! It's nice to meet you. How can I assist you today?"

In [20]:
conversation.predict(input="What is 1+1?")

'1+1 is equal to 2.'

In [21]:
conversation.predict(input="What is my name?")

"I'm sorry, but I don't have access to personal information."

### ConversationTokenBufferMemory

In [22]:
# !pip install tiktoken

In [23]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import AzureOpenAI
llm = AzureOpenAI(
    openai_api_version=OPENAI_API_VERSION,
    deployment_name=OPENAI_DEPLOYMENT,
    temperature=0.0
)

In [24]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=30)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [25]:
memory.load_memory_variables({})

{'history': 'AI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

### ConversationSummaryMemory

In [26]:
from langchain.memory import ConversationSummaryBufferMemory

In [27]:
llm = AzureChatOpenAI(
    openai_api_version=OPENAI_API_VERSION,
    deployment_name=OPENAI_DEPLOYMENT,
    temperature=0.0
)

In [28]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [29]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings. The human asks about the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the power of LangChain and suggests bringing a laptop to showcase the latest LLM demo during the lunch meeting.'}

In [30]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [31]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings. The human asks about the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the power of LangChain and suggests bringing a laptop to showcase the latest LLM demo during the lunch meeting.
Human: What would be a good demo to show?
AI:

> Finished chain.


'A good demo to show during the lunch meeting would be the latest Language Learning Model (LLM) demo on the LangChain project. The LLM demo showcases the advanced natural language processing capabilities of LangChain, allowing users to interact with the system using conversational language. It can understand and respond to complex queries, provide accurate translations, and even generate human-like text. This demo would impress the customer and demonstrate the potential of LangChain in various applications.'

In [32]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings. The human asks about the schedule for the day. The AI provides a detailed schedule, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI emphasizes the power of LangChain and suggests bringing a laptop to showcase the latest LLM demo during the lunch meeting. The human asks what would be a good demo to show.\nAI: A good demo to show during the lunch meeting would be the latest Language Learning Model (LLM) demo on the LangChain project. The LLM demo showcases the advanced natural language processing capabilities of LangChain, allowing users to interact with the system using conversational language. It can understand and respond to complex queries, provide accurate translations, and even generate human-like text. This demo would impress the customer and demonstrate the potential of LangChain in various applications.'}